<div style="text-align: right"><b>Daniel Stevens</b><br><i>Socorro Esther Dominguez Vidana<i/><br><i>Programming in Python for Data Science - Aug 2021</i><br><i>25 Oct 2021</i></div>

# **Top Performing Movie Genre Director: A Disney Dataset Exploratory Analysis**



## **Introduction**

<i>"The Walt Disney Company, commonly just Disney (/ˈdɪzni/), is an American multinational entertainment and media conglomerate headquartered at the Walt Disney Studios complex in Burbank, California...</i>

<i>The company established itself as a leader in the American animation industry before diversifying into live-action film production, television, and theme parks."</i> ([Wikipedia contributors](http://en.wikipedia.org/wiki/The_Walt_Disney_Company)).

In July of 2021, BBC News reported that female actor Scarlett Johansson was suing The Walt Disney Company for breach of contract over the simultaneous release of her latest film *Black Widow* in both theaters and on Disney's streaming service, Disney+. Johansson claimed that her contract stipulated an exclusively theatrical release and that she was damaged through loss of earnings due to breach of this term as her salary was largely based on box office performance (BBC News, [“Scarlett Johansson Sues Disney over Streaming of Black Widow”](http://www.bbc.com/news/business-58757748)). Disney first refuted her claims, but the suit was eventually settled in October of 2021, the terms of which were undisclosed (BBC News, [“Black Widow: Disney and Scarlett Johansson Settle Lawsuit”](http://www.bbc.com/news/world-us-canada-58017445)).

This lawsuit highlights the potential earning power actors have when contracts are properly negotiated, and the lengths that large movie studios will go to in order to hold on to earnings. By understanding the potential box office performance of a film and contracts that have been agreed upon in the past with other actors on similar projects, actors that are in high demand to star in these films and can negotiate more advantageous contract terms with high-powered movies studios to maximize their personal earnings.

The dataset we will be examining is the Disney dataset, compiled by Kelly Garrett and Lichen Zhen (2017). The data can be found on the [data.world](http://data.world/kgarrett/disney-character-success-00-16) website.

**Through examination of the Disney dataset, we hope to answer the following questions:**
- *What is the best performing movie genre as measured by average inflation-adjusted gross revenue?*
- *Who is the director who has directed multiple movies within the genre and performed the best based on average inflation-adjusted gross revenue?*
- *What actors has this director worked with in the past within the genre, and what kinds of characters did they play?*

Answers to these questions could help actors determine what genre it is best to break into and who is the best director to work with within that genre, when making money is the goal. Knowing who has starred in these types of films would allow an actor to know who their competition is, or be able to consult with these actors and their representatives regarding past contract terms to ensure that the best deal is being made.

Our hypothesis is that the `Musical` genre will have the highest average inflation-adjusted gross revenue as this is the genre that The Walt Disney Company rose to popularity with.

**The Disney dataset consists of 5 tables stored as .csv files:**
- `disney_movies_total_gross.csv` - lists the total gross revenue and total inflation adjusted revenue for movie titles in USD, along with their release date, rating and genre.
- `disney_revenue_1991-2016.csv` - lists the total gross revenue of various divisions of The Walt Disney company by year in millions USD.
- `disney-characters.csv` - lists disney characters, the movie they appear in, their character type (hero or villain), the song associated with the movie and the movie release date.
- `disney-director.csv` - lists movies and their associated directors (limited to a single director per movie, even when there was more than one)   
- `disney-voice-actors.csv` - lists disney movie characters, their voice actor/s and the movie they appear in.


For this analysis, we require all data sets except for `disney_revenue_1991-2016.csv`.


<hr>

## **Methods & Results**

First we will import the libraries and functions necessary for our analysis.

In [1]:
import pandas as pd
import altair as alt
from currency_cleaner import currency_cleaner
from split_melt import split_melt

Next, we will read in the required csv files, creating a dataframe for each.

In [2]:
disney_movies_total_gross = pd.read_csv('../data/disney_movies_total_gross.csv')
disney_director = pd.read_csv('../data/disney-director.csv')
disney_voice_actors = pd.read_csv('../data/disney-voice-actors.csv')
disney_characters = pd.read_csv('../data/disney-characters.csv')

### **Data Overview**
Now that our data has been read in, we will look at each dataframe, familiarize ourselves and summarize their characteristics.

#### **`disney_movies_total_gross`**

In [3]:
disney_movies_total_gross.head(2)

,movie_title,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross
0,Snow White and the Seven Dwarfs,"Dec 21, 1937",Musical,G,"$184,925,485","$5,228,953,251"
1,Pinocchio,"Feb 9, 1940",Adventure,G,"$84,300,000","$2,188,229,052"


In [4]:
disney_movies_total_gross.shape

(579, 6)

In [5]:
disney_movies_total_gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579 entries, 0 to 578
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   movie_title               579 non-null    object
 1   release_date              579 non-null    object
 2   genre                     562 non-null    object
 3   MPAA_rating               523 non-null    object
 4   total_gross               579 non-null    object
 5   inflation_adjusted_gross  579 non-null    object
dtypes: object(6)
memory usage: 27.3+ KB


The `disney_movies_total_gross` dataframe consists of 579 rows with 6 columns, all of dtype `object`. The columns consist of two nominal variables (`movie_title` and `genre`), one temporal variable (`release_date`), one ordinal variable (`MPAA_rating`) and two quantitative variables (`total_gross` and `inflation-adjusted gross`, representing the movie's total gross revenue and inflation-adjusted gross revenue respectively, in USD). Both the genre and the MPAA_rating have missing values.

In [6]:
disney_movies_total_gross.isna().sum()

movie_title                  0
release_date                 0
genre                       17
MPAA_rating                 56
total_gross                  0
inflation_adjusted_gross     0
dtype: int64

There are 17 observations missing a value for `genre`, and 56 observations missing a value for `MPAA_rating`.

#### **`disney_director`**

In [7]:
disney_director.head(2)

,name,director
0,Snow White and the Seven Dwarfs,David Hand
1,Pinocchio,Ben Sharpsteen


In [8]:
disney_director.shape

(56, 2)

In [9]:
disney_director.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      56 non-null     object
 1   director  56 non-null     object
dtypes: object(2)
memory usage: 1.0+ KB


The `disney_director` dataframe consists of 56 rows and 2 columns of the dtype `object`. The two columns each represent a single nominal variable, the `name` column containing a movie title, and the `director` column containing the name of one of the movie's directors. There are no missing values.

#### **`disney_voice_actors`**

In [10]:
disney_voice_actors.head(2)

,character,voice-actor,movie
0,Abby Mallard,Joan Cusack,Chicken Little
1,Abigail Gabble,Monica Evans,The Aristocats


In [11]:
disney_voice_actors.shape

(935, 3)

In [12]:
disney_voice_actors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 935 entries, 0 to 934
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   character    935 non-null    object
 1   voice-actor  935 non-null    object
 2   movie        935 non-null    object
dtypes: object(3)
memory usage: 22.0+ KB


The `disney_voice_actors` dataframe consists of 935 rows and 3 columns of dtype `object`. The three columns each represent a single nominal variable, the `movie` column containing a movie title, the `voice-actor` column containing the name of a voice actor in the movie, and the `character` column containing the name of the character the actor played in the movie. There are no missing values.

#### **`disney_characters`**

In [13]:
disney_characters.head(2)

,movie_title,release_date,hero,villian,song
0,\nSnow White and the Seven Dwarfs,"December 21, 1937",Snow White,Evil Queen,Some Day My Prince Will Come
1,\nPinocchio,"February 7, 1940",Pinocchio,Stromboli,When You Wish upon a Star


In [14]:
disney_characters.shape

(56, 5)

In [15]:
disney_characters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movie_title   56 non-null     object
 1   release_date  56 non-null     object
 2   hero          52 non-null     object
 3   villian       46 non-null     object
 4   song          47 non-null     object
dtypes: object(5)
memory usage: 2.3+ KB


The `disney_characters` dataframe contains 56 rows and 5 columns all of dtype `object`. There are 2 'untidy' columns, `hero` and `villian` which represent a single nominal variable of character type, and contained between these two columns is a single nominal variable which is the character name. There are 2 additional nominal variable columns, `movie_title` which contains the title of the movie, and `song` which contains the name of a song from the movie. The final column, `release_date` contains a temporal variable, which represents the date that the movie was released. The variables `hero`, `villain` and `song` all have missing values.

In [16]:
disney_characters.isna().sum()

movie_title      0
release_date     0
hero             4
villian         10
song             9
dtype: int64

There are 4 missing values in the `hero` column, 10 in the `villian` column, and 9 in the `song` column.
<hr>

## **Cleanliness and Tidiness of Data**

Now that we have a sense of the overall shape and content of our data, let's check how clean and tidy it is.

#### **Check for Duplicate Observations**
Let's check our `disney_movies_total_gross` data set to make sure all observations are unique. The `movie_title` and `release_date` columns uniquely identify each observation, so we can use this subset of variables to check for duplicates.

In [17]:
gross_duplicate_check = disney_movies_total_gross.duplicated(subset = ["movie_title","release_date"], keep = "first")
gross_duplicates = gross_duplicate_check[gross_duplicate_check == True]
gross_duplicates

Series([], dtype: bool)

An empty series was returned, so we can reasonably conclude that there are no duplicate observations.

The same general method was applied `disney_director`, `disney_voice_actors` and `disney_characters`, and no duplicate observations were found.

#### **Check Tidiness of Data**

`disney_movies_total_gross` and `disney_director` are tidy datasets, while `disney_characters` and `disney_voice_actors` are not.

**`disney_characters` violates 2 of the tidy data conditions:**
- The `hero` and `villian` columns are not variables, they are observational values of a variable we will call `character_type`. Since the `character_type` variable is split between the `hero` and `villian` columns, this violates the tidy data condition that each variable must be a single column.
- The `hero` and `villian` columns also contain single cells with multiple values. For example the `hero` column contains the entry 'Lilo and Stitch' which is not a single value, but two values.

We will melt the `hero` and `villian` columns into the variable column `character_type` and the values of the columns will be populated to a column called `character_name`.

**`disney_voice_actors` violates 1 of the tidy data conditions in two of its columns:**
- The `character` column contains cells with more than one value
- The `voice-actor` column contains cells with more than one value

We will have to split these values carefully, as the all values in some cells can be attributed to a single character or actor (e.g. actor plays multiple characters in the movie), while in other cells, only a single value can be attributed to a single actor or character (e.g. two actor names and two character names are provided, and their positions within their respective cells correspond to which values between the two cells belong together.)

#### **Check Cleanliness of Categorical Variables**
The categorical variables contained in this data set will be used for grouping. It is important that the values for this variables are consistent with each other so that appropriate observations can be grouped. Let's check the cleanliness of our categorical variables.

In [18]:
print(disney_movies_total_gross.loc[:, 'genre'].nunique())
disney_movies_total_gross.loc[:, 'genre'].unique()

12


array(['Musical', 'Adventure', 'Drama', 'Comedy', nan, 'Action', 'Horror',
       'Romantic Comedy', 'Thriller/Suspense', 'Western', 'Black Comedy',
       'Documentary', 'Concert/Performance'], dtype=object)

There are **12 unique movie genres** : Musical, Adventure, Drama, Comedy, Action, Horror, Romantic Comedy, Thriller/Suspense, Western, Black Comedy, Documentary and Concert/Performance. There are also NaN values, as noted earlier.

In [19]:
print(disney_movies_total_gross.loc[:, 'MPAA_rating'].nunique())
disney_movies_total_gross.loc[:, 'MPAA_rating'].unique()

5


array(['G', nan, 'Not Rated', 'PG', 'R', 'PG-13'], dtype=object)

There are **5 unique ratings**: G, Not Rated, PG, R and PG-13. There are also NaN values, as noted earlier.

The categorical variables in `disney_movies_total_gross` are consistent, therefore no cleansing is necessary.
`disney_characters` also contains categorical variables in the form of the untidy coulumns `hero` and `villian`. These columns will be melted through the data cleaning process, and thus they will remain consistent.

#### **Check Variable Dtypes**

**As noted in the Data Overview:**
- all columns in `disney_movies_total_gross` are of dtype `object`. For proper analysis, the temporal variable column `release_date` must be transformed into the `datetime` dtype, and the quanititative variable columns `total_gross` and `inflation_adjusted_gross` must be transformed into the `float` dtype.
- all columns in `disney_characters` are of dytpe `object`. For proper anaysis, the temporal variable column `release_date` must be transformed into the `datetime` dtype.
- all other columns in the datasets are of appropriate dtype for analysis.

## **Data Cleansing**

### **`disney_movies_total_gross`**
First we will address the `release_date` column dtype by converting it to `datetime`.

In [20]:
movies_gross_clean = disney_movies_total_gross.assign(
    release_date=pd.to_datetime(disney_movies_total_gross["release_date"])
)
movies_gross_clean.head(2)

,movie_title,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross
0,Snow White and the Seven Dwarfs,1937-12-21,Musical,G,"$184,925,485","$5,228,953,251"
1,Pinocchio,1940-02-09,Adventure,G,"$84,300,000","$2,188,229,052"


In [21]:
movies_gross_clean.loc[:,'release_date'].dtypes

dtype('<M8[ns]')

Next, we will employ the imported function <b><i>currency_cleaner( )</i></b> to convert the total gross and inflation-adjusted gross columns to dtype float. <b><i>currency_cleaner( )</i></b> removes the specified currency symbol and any commas or spaces from a string value representing a currency amount, and casts the string value to a float.

In [22]:
movies_gross_clean = currency_cleaner(
    movies_gross_clean, ["total_gross", "inflation_adjusted_gross"]
)
movies_gross_clean.head(2)

,movie_title,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross
0,Snow White and the Seven Dwarfs,1937-12-21,Musical,G,184925485.0,5.228953e+09
1,Pinocchio,1940-02-09,Adventure,G,84300000.0,2.188229e+09


In [23]:
movies_gross_clean.loc[:,['total_gross','inflation_adjusted_gross']].dtypes

total_gross                 float64
inflation_adjusted_gross    float64
dtype: object

### **`disney_characters`**

First we will address the `release_date` column by converting the dtype to `datetime`, and we will clean the string values of the `movie_title` and `song` columns by removing '\n'.

In [24]:
characters_clean = disney_characters.assign(release_date = pd.to_datetime(disney_characters['release_date']),
                                             movie_title = disney_characters['movie_title'].str.strip('\n'),
                                             song = disney_characters['song'].str.strip('\n')
                                            )

Next, we will tidy the dataframe by employing the imported function <b><i>split_melt( )</i></b>. This function splits the string values in the columns specified, expands the result, concatenates the expanded columns to the ID columns of the original dataframe and then melts the expanded columns into the concatenated dataframe. If more than one column is split, a new column is created and populated with each melted value's original column name (see `character_type` column below).

In [25]:
# correct `villian` spelling to villain
characters_clean = characters_clean.rename(columns={"villian": "villain"})

characters_clean = split_melt(
    characters_clean,
    ["movie_title", "release_date", "song"],
    ["hero", "villain"],
    " and ",
    "character_type",
    "character_name",
)

characters_clean.head(2)

,movie_title,release_date,song,character_type,character_name
0,Snow White and the Seven Dwarfs,1937-12-21,Some Day My Prince Will Come,hero,Snow White
1,Pinocchio,1940-02-07,When You Wish upon a Star,hero,Pinocchio


### **`disney_voice_actors`**

The `voice-actor` column and `character` column both contain cells with multiple values, however, values in the cells do not all relate to each other in the same way. Let's loop through each delimiter for each actor and character value and split the value on the delimiter it contains.

In [26]:
voice_actors_clean = disney_voice_actors.rename(columns={"voice-actor": "voice_actor"})

delimiters = [" & ", " and ", "/", "; "]
columns = ["character", "voice_actor"]

# If delimiter in actor/character name, split value on delimiter
for column in columns:
    for index, name in voice_actors_clean[column].items():
        for delimiter in delimiters:
            if delimiter in name:
                voice_actors_clean.loc[index, column] = name.split(delimiter)
                break
            else:  # If delimiter not in actor/character name, create list of duplicated name.
                voice_actors_clean.loc[index, column] = [name, name]
voice_actors_clean.head(2)


,character,voice_actor,movie
0,"[Abby Mallard, Abby Mallard]","[Joan Cusack, Joan Cusack]",Chicken Little
1,"[Abigail Gabble, Abigail Gabble]","[Monica Evans, Monica Evans]",The Aristocats


Next, we will expand the split columns by casting them to a list and assigning the resulting list of lists to new columns in the dataframe.

In [27]:
# Expand list values of actor names into 2 columns and add to dataframe
voice_actors_clean[["voice_actor1", "voice_actor2"]] = pd.DataFrame(
    voice_actors_clean.loc[:, "voice_actor"].tolist(), index=voice_actors_clean.index
)
voice_actors_clean.head(2)


,character,voice_actor,movie,voice_actor1,voice_actor2
0,"[Abby Mallard, Abby Mallard]","[Joan Cusack, Joan Cusack]",Chicken Little,Joan Cusack,Joan Cusack
1,"[Abigail Gabble, Abigail Gabble]","[Monica Evans, Monica Evans]",The Aristocats,Monica Evans,Monica Evans


In [28]:
# Expand list values of character names into 2 columns and add to dataframe
voice_actors_clean[["char1", "char2"]] = pd.DataFrame(
    voice_actors_clean.loc[:, "character"].tolist(), index=voice_actors_clean.index
)
voice_actors_clean.head(2)


,character,voice_actor,movie,voice_actor1,voice_actor2,char1,char2
0,"[Abby Mallard, Abby Mallard]","[Joan Cusack, Joan Cusack]",Chicken Little,Joan Cusack,Joan Cusack,Abby Mallard,Abby Mallard
1,"[Abigail Gabble, Abigail Gabble]","[Monica Evans, Monica Evans]",The Aristocats,Monica Evans,Monica Evans,Abigail Gabble,Abigail Gabble


Now let's pair each actor with their character and concatenate the actor-character pairs vertically.

In [29]:
# Select only actor 1 and character 1 with movie, rename columns
voice_actor_1 = voice_actors_clean.loc[:, ["movie", "voice_actor1", "char1"]].rename(
    columns={
        "movie": "movie_title",
        "voice_actor1": "voice_actor",
        "char1": "character_name",
    }
)

In [30]:
# Select only actor 2 and character 2 with movie, rename columns
voice_actor_2 = voice_actors_clean.loc[:, ["movie", "voice_actor2", "char2"]].rename(
    columns={
        "movie": "movie_title",
        "voice_actor2": "voice_actor",
        "char2": "character_name",
    }
)


In [31]:
# Concatenate dataframes vertically
voice_actor_full = pd.concat([voice_actor_1, voice_actor_2], axis=0)
voice_actor_full.shape

(1870, 3)

This method results in some duplicate observations. Let's get rid of them.

In [32]:
# Remove duplicated observations
voice_actors_clean = voice_actor_full[
    ~voice_actor_full.duplicated()
].reset_index(drop=True)
voice_actors_clean.shape

(996, 3)

### `disney_director`

The `name` column in the `disney_director` dataframe is ambiguous. Let's rename this column to `movie_title` so it is consistent with the rest of the dataframes.

In [33]:
director_clean = disney_director.rename(columns ={'name':'movie_title'})
director_clean.columns

Index(['movie_title', 'director'], dtype='object')

<hr>

### Data Analysis
We would like to investigate the performace of movies based on movie genre, measured by the movie's inflation-adjusted gross revenue. The inflation-adjusted gross revenue is the best measure of movie performance as it allows us to compare movies released at different times on an equal footing. Average will be used as not all genres contain the same number of movies. We will drop the `total_gross_M` and `MPAA_rating` columns as they are not relevant to our analysis. We will also drop rows with NaN values for genre as these are unable provide the required information.

In [34]:
genre_gross = (
    movies_gross_clean.drop(columns=["MPAA_rating", "total_gross"])
    .dropna(subset=["genre"])
    .sort_values(by="inflation_adjusted_gross", ascending=False)
)
genre_gross.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 562 entries, 0 to 29
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   movie_title               562 non-null    object        
 1   release_date              562 non-null    datetime64[ns]
 2   genre                     562 non-null    object        
 3   inflation_adjusted_gross  562 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 22.0+ KB


We now have 562 observations with no missing values. How many movies are contained in each genre? Let's count the number of each unique value occurring in the `genre` column.

In [35]:
genre_gross['genre'].value_counts()

Comedy                 182
Adventure              129
Drama                  114
Action                  40
Thriller/Suspense       24
Romantic Comedy         23
Musical                 16
Documentary             16
Western                  7
Horror                   6
Black Comedy             3
Concert/Performance      2
Name: genre, dtype: int64

`Comedy` has the most titles, followed by `Adventure` and `Drama`. Let's use a box plot to visualize the distribution of inflation-adjusted gross revenue for each genre.

In [36]:
genre_gross_boxplot = (
    alt.Chart(genre_gross)
    .mark_boxplot(extent=3, size=30)
    .encode(
        x=alt.X("genre:N", title="Movie Genre"),
        y=alt.Y(
            "inflation_adjusted_gross:Q",
            title="Inflation-Adjusted Gross Revenue (millons)",
        ),
    )
    .properties(
        title="Figure 1: Distribution of Inflation-Adjusted Gross Revenue by Genre",
        width=500,
        height=500,
    )
).configure_axis(labelFontSize=12, titleFontSize=12)

genre_gross_boxplot


alt.Chart(...)

Upon inspection of figure 1 above, some major outliers are noted for several genres, most notably in the `Musical` category. Let's create a scatter plot of inflation-adjusted revenue vs year to visualize how these outliers may relate to release date.

In [37]:
genre_gross_scatter = (
    (
        alt.Chart(genre_gross)
        .mark_circle(opacity=0.3)
        .encode(
            x=alt.X("release_date:T", title="Movie Release Date"),
            y=alt.Y(
                "inflation_adjusted_gross:Q",
                title="Inflation-Adjusted Gross Revenue (USD)",
            ),
        )
    )
    .properties(
        title="Figure 2: Inflation-Adjusted Gross Revenue of Movies by Year",
        width=500,
        height=300,
    )
    .configure_axis(labelFontSize=12, titleFontSize=12)
)
genre_gross_scatter


alt.Chart(...)

It appears that movies with earlier release dates have  considerably higer inflation-adjusted gross revenue on average. Whether these outliers are real or due to an experimental/calculation error is unknown as the method for obtaining the inflation-adjusted gross revenue was not detailed by the authors of the data. In researching the movie with the highest inflation-adjusted gross revenue, *Snow White and the Seven Dwarfs*, we discovered that the total box office gross revenue for this movie is in fact 184.93 M as our dataset reports, however, this is the combined total for the years 1937, 1983, 1987 and 1993 (“Snow White and the Seven Dwarfs”). If the authors of the dataset assumed that this total was achieved exclusively in 1937, this woud cause an error in the inflation-adjusted gross revenue calculation. To avoid these outliers from skewing our data, let's focus on the years 1980 - 2016 where a majority of the data is present and appears consistent. It is less likely that these values are problematic as the more recent a movie is, the less likely it is to have been re-released, and if it has, any miscalculation of inflation-adjusted gross due to revenue being attributed to the incorrect year would be less severe.

In [38]:
genre_gross_mean = (
    genre_gross[genre_gross["release_date"].dt.year >= 1980]
    .groupby(by="genre")
    .agg({"inflation_adjusted_gross": "mean"})
    .sort_values(by="inflation_adjusted_gross", ascending=False)
).reset_index()

# Convert to millions and round to 2 decimal places
genre_gross_mean_M = genre_gross_mean.assign(
    inflation_adjusted_gross_M=(
        genre_gross_mean["inflation_adjusted_gross"] / 1_000_000
    ).round(2)
).drop(columns="inflation_adjusted_gross")

genre_gross_mean_M

,genre,inflation_adjusted_gross_M
0,Adventure,164.94
1,Action,137.47
2,Musical,98.06
3,Thriller/Suspense,89.65
4,Romantic Comedy,77.78
5,Comedy,75.19
6,Western,73.82
7,Concert/Performance,57.41
8,Drama,54.21
9,Black Comedy,52.24


From the table `genre_gross_mean_M`  above, we can see that the `Adventure` genre has the highest mean inflation-adjusted gross revenue for years 1980-2016 at 164.94 M, while the `Documentary` genre has the lowest at 12.72 M. Let's visualize this with a bar plot.

In [39]:
mean_bar_plot = (
    alt.Chart(genre_gross_mean_M)
    .mark_bar()
    .encode(
        x=alt.X("genre:N", title="Movie Genre", sort="-y"),
        y=alt.Y(
            "inflation_adjusted_gross_M:Q",
            title="Mean Infation-Adjusted Gross Revenue (millions USD)",
        )
    )
    .properties(
        title="Figure 3: Mean Inflation-Adjusted Gross Revenue by Genre",
        width=500,
        height=500,
    )
)

mean_bar_plot_text = mean_bar_plot.mark_text(dy=15, size= 12, color = 'white', 
).encode(x=alt.X("genre:N", title="Movie Genre", sort="-y"),
        y=alt.Y(
            "inflation_adjusted_gross_M:Q"),
         text='inflation_adjusted_gross_M:Q'
)

mean_bar_plot + mean_bar_plot_text

alt.LayerChart(...)

Let's find a director who has a proven track record in the `Adventure` genre for the target date range. We will first find a list of directors who have directed more than 1 Adventure movie between 1980 and 2016. We will then calculate the average inflation-adjusted gross revenue for the movies each director has directed in the target genre and timeframe, and determine which director has the highest average.

First, let's eliminate all the duplicate movie titles in the `Adventure` genre from the `genre_gross` dataframe i.e. the movie remakes. We are doing this to ensure that no movie is misattributed to a director, since the `director_clean` dataframe only specifies the title of the movie and not its release date.

In [40]:
adventure_titles = genre_gross[
    (genre_gross["genre"] == "Adventure")
    & (genre_gross["release_date"].dt.year >= 1980)
]
adventure_duplicate_titles_check = adventure_titles.duplicated(
    subset="movie_title", keep=False
)
adventure_duplicate_titles_check.value_counts()

False    122
True       2
dtype: int64

There are 2 duplicate values for `movie_title` in the Adventure genre of the `genre_gross` dataframe between 1980 and 2016. Let's eliminate these movies from the dataframe.

In [41]:
unique_adventure_titles = adventure_titles[~adventure_duplicate_titles_check]

Next, we will merge the `unique_adventure_titles` dataframe with the `director_clean` dataframe to determine who directed each movie.

In [42]:
unique_adventure_titles_director = unique_adventure_titles.loc[
    :, ["movie_title", "inflation_adjusted_gross"]
].merge(
    director_clean,
    left_on="movie_title",
    right_on="movie_title",
    how="left",
    indicator=True,
)
unique_adventure_titles_director = unique_adventure_titles_director[
    unique_adventure_titles_director["_merge"] == "both"
].drop(columns="_merge")

Now that we have attributed each unique adventure title to a director, let's group by director, find the average inflation-adjusted gross revenue for their movies in the targe genre and timeframe, then filter for those that have directed more than one movie. We will express our resulting inflation-adjusted gross revenue in millions.

In [43]:
adventure_director_average = unique_adventure_titles_director.groupby(
    by="director"
).agg({"movie_title": "count", "inflation_adjusted_gross": "mean"})
repeat_adventure_director_average = (
    adventure_director_average[adventure_director_average["movie_title"] > 1]
    .sort_values(by="inflation_adjusted_gross", ascending=False)
    .reset_index()
)
repeat_adventure_director_average = repeat_adventure_director_average.assign(
    inflation_adjusted_gross_M=(
        repeat_adventure_director_average["inflation_adjusted_gross"] / 1_000_000
    ).round(2)
).drop(columns="inflation_adjusted_gross")
repeat_adventure_director_average

,director,movie_title,inflation_adjusted_gross_M
0,Chris Buck,2,349.45
1,Mike Gabriel,2,165.08
2,Gary Trousdale,2,158.09
3,Mark Dindal,2,157.37
4,Ron Clements,6,146.16
5,Stephen J. Anderson,2,74.12


Let's visualize the average inflation-adjusted gross revenue for repeat adventure movie directors using a bar chart.

In [44]:
repeat_adventure_director_average_chart = (
    alt.Chart(repeat_adventure_director_average)
    .mark_bar()
    .encode(
        x=alt.X("director:N", title="Repeat Adventure Director", sort="-y"),
        y=alt.Y(
            "inflation_adjusted_gross_M:Q",
            title="Average Inflation-Adjusted Gross Revenue of Adventure Movies (millions USD)",
        ),
    )
    .properties(
        title="Figure 4: Average Inflation-Adjusted Gross Movie Revenue of Repeat Adventure Movie Directors",
        width=500,
        height=500,
    )
)

repeat_adventure_director_average_chart_text = (
    repeat_adventure_director_average_chart.mark_text(
        dy=15,
        size=12,
        color="white",
    ).encode(
        x=alt.X("director:N", title="Repeat Adventure Director", sort="-y"),
        y=alt.Y("inflation_adjusted_gross_M:Q"),
        text="inflation_adjusted_gross_M:Q",
    )
)

repeat_adventure_director_average_chart + repeat_adventure_director_average_chart_text


alt.LayerChart(...)

Based on the figure 4 above, director Chris Buck is the repeat adventure movie director whose adventure movies have had the highest average inflation-adjusted gross revenue at 349.45 M.

What actors has Chris Buck worked on Adventure movies with in the past and what types of roles did they play in these films? To determine this, we will filter the `unique_adventure_titles_director` dataframe to only contain Chris Buck directed movies and merge this dataframe with the `characters_clean` dataframe and then with the `voice_actors_clean` dataframe.

In [45]:
chris_buck_movie_characters = (
    unique_adventure_titles_director[
        unique_adventure_titles_director["director"] == "Chris Buck"
    ]
    .loc[:, ["movie_title"]]
    .merge(
        characters_clean.loc[
            :, ["movie_title", "character_name", "character_type"]
        ],
        left_on="movie_title",
        right_on="movie_title",
        how="left",
        indicator=True,
    )
)


In [46]:
chris_buck_movie_character_actors = chris_buck_movie_characters.merge(
    voice_actors_clean,
    left_on=["movie_title", "character_name"],
    right_on=["movie_title", "character_name"],
    how="left",
    indicator="_merge2",
)
chris_buck_movie_character_actors = chris_buck_movie_character_actors.drop(
    columns=["_merge", "_merge2"]
)
chris_buck_movie_character_actors

,movie_title,character_name,character_type,voice_actor
0,Frozen,Elsa,hero,Idina Menzel
1,Frozen,Prince Hans,villain,Santino Fontana
2,Tarzan,Tarzan,hero,Tony Goldwyn
3,Tarzan,Tarzan,hero,Alex D. Linz
4,Tarzan,Clayton,villain,Brian Blessed


From the table `chris_buck_movie_character_actors` above, we can see that 5 different actors worked with Chris Buck on his two adventure movies Tarzan and Frozen. Three played the hero, and two played the villain. Below is a summary of Chris Buck's adventure films.

In [47]:
chris_buck_adventure_summary = chris_buck_movie_character_actors.merge(
    movies_gross_clean.loc[
        :,
        [
            "movie_title",
            "release_date",
            "MPAA_rating",
            "genre",
            "total_gross",
            "inflation_adjusted_gross",
        ],
    ],
    left_on="movie_title",
    right_on="movie_title",
    how="left",
    indicator=True,
)
chris_buck_adventure_summary = chris_buck_adventure_summary.loc[
    :,
    [
        "release_date",
        "movie_title",
        "genre",
        "MPAA_rating",
        "total_gross",
        "inflation_adjusted_gross",
        "voice_actor",
        "character_name",
        "character_type",
    ],
].sort_values(by="release_date")
chris_buck_adventure_summary = (
    chris_buck_adventure_summary.assign(
        inflation_adjusted_gross_M=(
            chris_buck_adventure_summary["inflation_adjusted_gross"] / 1_000_000
        ).round(2),
        total_gross_M=(chris_buck_adventure_summary["total_gross"] / 1_000_000).round(
            2
        ),
    )
    .drop(columns=["total_gross", "inflation_adjusted_gross"])
    .reset_index(drop=True)
)
chris_buck_adventure_summary


,release_date,movie_title,genre,MPAA_rating,voice_actor,character_name,character_type,inflation_adjusted_gross_M,total_gross_M
0,1999-06-16,Tarzan,Adventure,G,Tony Goldwyn,Tarzan,hero,283.9,171.09
1,1999-06-16,Tarzan,Adventure,G,Alex D. Linz,Tarzan,hero,283.9,171.09
2,1999-06-16,Tarzan,Adventure,G,Brian Blessed,Clayton,villain,283.9,171.09
3,2013-11-22,Frozen,Adventure,PG,Idina Menzel,Elsa,hero,415.0,400.74
4,2013-11-22,Frozen,Adventure,PG,Santino Fontana,Prince Hans,villain,415.0,400.74


## **Discussion**

From our analysis, we determined that the `Adventure` genre has the highest mean inflation-adjusted gross revenue (164.94 M) between 1980 and 2016. This is contrary to our hypothesis that the `Musical` genre would be the best performer on average. In actuality, `Musical[s]` (98.06 M) ranked third in mean inflation-adjusted gross revenue behind `Adventure` and `Action` (137.47 M) but considerably higher than genres such as `Documentary` (12.72 M) and `Horror` (23.41 M). This could be due to the rise in popularity of the `Adventure` genre in recent decades. 

The director who has directed multiple movies in the `Adventure` genre with the highest average inflation-adjusted gross revenue for these movies is Chris Buck (349.45 M), the director of *Tarzan* (1999, 283.9 M) and *Frozen* (2013, 415.0 M) which was the highest grossing movie of 2013 (“2013 Worldwide Box Office”). Buck's average inflation-adjusted gross revenue in the `Adventure genre` is more than twice the average of the second place director, Mike Gabriel (165.08 M), who directed *Pocahontas* and *The Rescuers Down Under*.

Buck worked with Tony Goldwyn (Tarzan, hero), Alex D. Linz (Tarzan, hero) and Brian Blessed (Clayton, villain) on *Tarzan* and Idina Menzel (Elsa, hero) and Santino Fontana (Prince Hans, villain) on *Frozen*.

From these results, it would seem that up-and-coming actors would be best served by breaking into the `Adventure` genre. If an opportunity came up to work with Chris Buck, the movie could be a hit based off his past performance in the genre. Under this circumstance, it may be more financially advantageous to fight for a higher percentage on the box office revenue and be less focused on the initial payout. If the studio were offering a contract, it might be a good idea to get the same representation as Idina Menzel or Tony Goldwyn, as they have experience representing actors in successful movies within the Adventure genre directed by Chris Buck and would be aware of past deals. 

While Chris Buck is the top repeat director in the Adventure genre, do his movies have the highest inflation-adjusted gross revenue across all genres? If not, who does? Which director simultaneously had a huge hit, and a huge flop? (i.e. the greatest range in inflation-adjusted gross revenue). Who has starred most frequently in movies by Disney, and how much have these movies earned the studio? Such questions can be easily answered in future analyses of the cleaned Disney dataset.
<hr>

## **References**

“2013 Worldwide Box Office.” Box Office Mojo, 2021, www.boxofficemojo.com/year/world/2013.

BBC News. “Black Widow: Disney and Scarlett Johansson Settle Lawsuit.” BBC News, 1 Oct. 2021, www.bbc.com/news/business-58757748.

BBC News. “Scarlett Johansson Sues Disney over Streaming of Black Widow.” BBC News, 30 July 2021, www.bbc.com/news/world-us-canada-58017445.

“Disney Character Success - Dataset by Kgarrett.” Data.World, 15 Oct. 2021, [data.world/kgarrett/disney-character-success-00-16.](http://data.world/kgarrett/disney-character-success-00-16)

“Snow White and the Seven Dwarfs.” Box Office Mojo, 2021, www.boxofficemojo.com/title/tt0029583/?ref_=bo_gr_ti.

Wikipedia contributors. “The Walt Disney Company.” Wikipedia, 18 Oct. 2021, [en.wikipedia.org/wiki/The_Walt_Disney_Company.](http://en.wikipedia.org/wiki/The_Walt_Disney_Company)
